# NVIDIA iQuHACK 2026: Phase 2 Submission
**Solo:** Prithvinesan Thanigainesan
**Project:** GPU Accelerated MTS for LABS
**Hardware:** NVIDIA L4

In [1]:
import cudaq
import cupy as cp
import numpy as np

cudaq.set_target("nvidia")

print(f"CUDA-Q Target: {cudaq.get_target().name}")
!nvidia-smi --query-gpu=name,memory.total --format=csv

CUDA-Q Target: nvidia
name, memory.total [MiB]
NVIDIA L4, 23034 MiB


In [2]:
def labs_energy_gpu(sequences):
    pop_size, N = sequences.shape
    energy = cp.zeros(pop_size)
    for k in range(1, N):
        c_k = cp.sum(sequences[:, :N-k] * sequences[:, k:], axis=1)
        energy += c_k**2
    return energy

In [4]:
'''
import cupy as cp

def labs_energy_gpu(sequences):

    batch_size, N = sequences.shape
    energy = cp.zeros(batch_size)
    
    for k in range(1, N):
        c_k = cp.sum(sequences[:, :N-k] * sequences[:, k:], axis=1)
        energy += c_k**2
        
    return energy

# test
N = 40 
batch = 1000
test_sequences = cp.random.choice(cp.array([-1, 1]), size=(batch, N))

import time
start = time.time()
energies = labs_energy_gpu(test_sequences)
cp.cuda.Stream.null.synchronize() # Wait for GPU to finish
print(f"Time for {batch} sequences: {time.time()-start:.4f}s")
print(f"Best energy in this random batch: {cp.min(energies)}")
'''

Time for 1000 sequences: 0.3711s
Best energy in this random batch: 272.0


In [5]:
'''def gpu_hill_climb(best_seq, iterations=100):
    N = len(best_seq)
    current_seq = cp.asarray(best_seq)
    current_energy = labs_energy_gpu(current_seq.reshape(1, -1))[0]
    
    print(f"Starting Hill Climb. Initial Energy: {current_energy}")
    
    for i in range(iterations):
        neighbors = cp.tile(current_seq, (N, 1))
        flip_mask = cp.eye(N) * -2 + 1 
        neighbors = neighbors * flip_mask
        
        energies = labs_energy_gpu(neighbors)
        
        best_neighbor_idx = cp.argmin(energies)
        new_energy = energies[best_neighbor_idx]
        
        if new_energy < current_energy:
            current_energy = new_energy
            current_seq = neighbors[best_neighbor_idx]
            if i % 10 == 0:
                print(f"Iteration {i}: New Best Energy = {current_energy}")
        else:
            print(f"Local optimum reached at iteration {i}.")
            break
            
    return cp.asnumpy(current_seq), float(current_energy)

best_idx = cp.argmin(energies)
best_initial_seq = test_sequences[best_idx]

refined_seq, final_energy = gpu_hill_climb(best_initial_seq)
print(f"\nFinal Refined Energy: {final_energy}")'''

Starting Hill Climb. Initial Energy: 272.0
Iteration 0: New Best Energy = 236.0
Local optimum reached at iteration 3.

Final Refined Energy: 180.0


In [6]:
def gpu_tabu_search(initial_seq, iterations=500, tabu_tenure=10):
    N = len(initial_seq)
    current_seq = cp.asarray(initial_seq)
    current_energy = labs_energy_gpu(current_seq.reshape(1, -1))[0]
    
    best_seq = cp.copy(current_seq)
    best_energy = current_energy
    
    tabu_list = cp.full(tabu_tenure, -1)
    tabu_idx = 0

    print(f"Starting Tabu Search. Baseline Energy: {best_energy}")

    for i in range(iterations):
        neighbors = cp.tile(current_seq, (N, 1))
        flip_mask = cp.eye(N) * -2 + 1 
        neighbors = neighbors * flip_mask
        
        energies = labs_energy_gpu(neighbors)
        
        penalty = cp.zeros(N)
        for t_bit in tabu_list:
            if t_bit != -1:
                if energies[t_bit] >= best_energy:
                    penalty[t_bit] = 1e9
        
        best_neighbor_idx = int(cp.argmin(energies + penalty))
        
        current_seq = neighbors[best_neighbor_idx]
        current_energy = energies[best_neighbor_idx]
        
        if current_energy < best_energy:
            best_energy = current_energy
            best_seq = cp.copy(current_seq)
            #print(f"Iteration {i}: New Global Best = {best_energy}")
            
        tabu_list[tabu_idx] = best_neighbor_idx
        tabu_idx = (tabu_idx + 1) % tabu_tenure
            
    return cp.asnumpy(best_seq), float(best_energy)

best_result_seq, final_best_energy = gpu_tabu_search(best_initial_seq, iterations=1000)

Starting Tabu Search. Baseline Energy: 272.0
Iteration 0: New Global Best = 236.0
Iteration 1: New Global Best = 208.0
Iteration 2: New Global Best = 180.0
Iteration 4: New Global Best = 148.0
Iteration 654: New Global Best = 140.0


In [7]:
N = 40
best_result_seq, final_best_energy = gpu_tabu_search(
    best_initial_seq, 
    iterations=10000, 
    tabu_tenure=8  
)

merit_factor = (N**2) / (2 * final_best_energy)
print(f"\nFinal best energy: {final_best_energy}")
print(f"Final merit factor: {merit_factor:.4f}")

Starting Tabu Search. Baseline Energy: 272.0
Iteration 0: New Global Best = 236.0
Iteration 1: New Global Best = 208.0
Iteration 2: New Global Best = 180.0
Iteration 4: New Global Best = 148.0
Iteration 958: New Global Best = 140.0
Iteration 2638: New Global Best = 132.0
Iteration 4081: New Global Best = 128.0
Iteration 5699: New Global Best = 120.0

Final Best Energy: 120.0
Final Merit Factor: 6.6667


In [13]:
print(best_global_energy)

108.0


In [9]:
'''import time

N = 40
num_starts = 200   
iters_per_start = 10000  
tenure = 12       

best_global_energy = float('inf')
best_global_seq = None

print("running")

for s in range(num_starts):
    seed_seq = cp.random.choice(cp.array([-1, 1]), size=N)
    res_seq, res_energy = gpu_tabu_search(seed_seq, iterations=iters_per_start, tabu_tenure=tenure)
    
    if res_energy < best_global_energy:
        best_global_energy = res_energy
        best_global_seq = res_seq
        print(f" -----> SEED {s}: New best found: Energy: {best_global_energy} | F: {(N**2)/(2*best_global_energy):.4f}")'''

running
Starting Tabu Search. Baseline Energy: 656.0
Iteration 0: New Global Best = 468.0
Iteration 1: New Global Best = 368.0
Iteration 2: New Global Best = 260.0
Iteration 3: New Global Best = 224.0
Iteration 4: New Global Best = 188.0
Iteration 12: New Global Best = 164.0
Iteration 107: New Global Best = 160.0
Iteration 236: New Global Best = 148.0
Iteration 5573: New Global Best = 120.0
Iteration 7410: New Global Best = 116.0
 -----> SEED 0: New best found: Energy: 116.0 | F: 6.8966
Starting Tabu Search. Baseline Energy: 544.0
Iteration 0: New Global Best = 404.0
Iteration 1: New Global Best = 320.0
Iteration 2: New Global Best = 292.0
Iteration 3: New Global Best = 256.0
Iteration 9: New Global Best = 248.0
Iteration 10: New Global Best = 212.0
Iteration 43: New Global Best = 192.0
Iteration 56: New Global Best = 188.0
Iteration 190: New Global Best = 180.0
Iteration 443: New Global Best = 176.0
Iteration 447: New Global Best = 168.0
Iteration 544: New Global Best = 164.0
Iteratio

KeyboardInterrupt: 

In [10]:
print(best_global_seq)

[-1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1. -1.  1.
 -1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1. -1.
  1.  1.  1.  1.]


In [ ]:
N = 40
num_starts = 200    
iters_per_start = 10000  
tenure = 12         

print(f"Current best: Energy {best_global_energy} (F: {(N**2)/(2*best_global_energy):.4f})")
print("started")

for s in range(num_starts):
    seed_seq = cp.random.choice(cp.array([-1, 1]), size=N)

    res_seq, res_energy = gpu_tabu_search(seed_seq, iterations=iters_per_start, tabu_tenure=tenure)
    
    if res_energy < best_global_energy:
        best_global_energy = res_energy
        best_global_seq = res_seq
        current_f = (N**2)/(2*best_global_energy)
        print(f"\n----> new best")
        print(f"Seed: {s} | Energy: {best_global_energy} | Merit Factor: {current_f:.4f}")
    
    if s % 10 == 0:
        print(f" (checking seeds {s}-{s+10}) ", end="")

print("\n200 Seeds completed.")

Current Record to Beat: Energy 116.0 (F: 6.8966)
started
Starting Tabu Search. Baseline Energy: 620.0
Iteration 0: New Global Best = 456.0
Iteration 1: New Global Best = 364.0
Iteration 2: New Global Best = 320.0
Iteration 3: New Global Best = 292.0
Iteration 4: New Global Best = 264.0
Iteration 5: New Global Best = 260.0
Iteration 6: New Global Best = 256.0
Iteration 8: New Global Best = 216.0
Iteration 9: New Global Best = 204.0
Iteration 12: New Global Best = 184.0
Iteration 17: New Global Best = 172.0
Iteration 34: New Global Best = 168.0
Iteration 35: New Global Best = 164.0
Iteration 679: New Global Best = 156.0
Iteration 964: New Global Best = 136.0
Iteration 6913: New Global Best = 132.0
 (checking seeds 0-10) Starting Tabu Search. Baseline Energy: 752.0
Iteration 0: New Global Best = 564.0
Iteration 1: New Global Best = 416.0
Iteration 2: New Global Best = 340.0
Iteration 3: New Global Best = 272.0
Iteration 4: New Global Best = 228.0
Iteration 7: New Global Best = 216.0
Itera

In [15]:
import numpy as np

final_energy = 108.0 
final_mf = (40**2) / (2 * final_energy)

print("=")
print("   LABS problem report")
print("=")
print(f"Sequence Length (N):   40")
print(f"Search Space Size:     2^40 (~1.1 Trillion)")
print(f"Algorithm:             GPU Parallelized tabu search")
print(f"Hardware:              NVIDIA L4 (24GB VRAM)")
print("-")
print(f"Final best energy:     {final_energy}")
print(f"Final merit factor:    {final_mf:.4f}")
print("-")
print(" My Best sequence:")
print(best_global_seq.astype(int).tolist())
print("=")

=
   LABS problem report
=
Sequence Length (N):   40
Search Space Size:     2^40 (~1.1 Trillion)
Algorithm:             GPU Parallelized tabu search
Hardware:              NVIDIA L4 (24GB VRAM)
-
Final best energy:     108.0
Final merit factor:    7.4074
-
 My Best sequence:
[1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1]
=
